In [3]:
import pandas as pd

In [4]:
train = pd.read_csv("train.csv")
train = train.iloc[:,:7].dropna()
train

C:\Users\parke\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (1,4,8,9) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,conv_id,utterance_idx,context,prompt,speaker_idx,utterance,selfeval
0,hit:0_conv:1,1,sentimental,I remember going to the fireworks with my best...,1,I remember going to see the fireworks with my ...,5|5|5_2|2|5
1,hit:0_conv:1,2,sentimental,I remember going to the fireworks with my best...,0,Was this a friend you were in love with_comma_...,5|5|5_2|2|5
2,hit:0_conv:1,3,sentimental,I remember going to the fireworks with my best...,1,This was a best friend. I miss her.,5|5|5_2|2|5
3,hit:0_conv:1,4,sentimental,I remember going to the fireworks with my best...,0,Where has she gone?,5|5|5_2|2|5
4,hit:0_conv:1,5,sentimental,I remember going to the fireworks with my best...,1,We no longer talk.,5|5|5_2|2|5
...,...,...,...,...,...,...,...
79193,hit:12424_conv:24848,5,sentimental,I found some pictures of my grandma in the att...,389,Yeah reminds me of the good old days. I miss ...,5|5|5_5|5|5
79194,hit:12424_conv:24849,1,surprised,I woke up this morning to my wife telling me s...,294,I woke up this morning to my wife telling me s...,5|5|5_5|5|5
79195,hit:12424_conv:24849,2,surprised,I woke up this morning to my wife telling me s...,389,Oh hey that's awesome! That is awesome right?,5|5|5_5|5|5
79196,hit:12424_conv:24849,3,surprised,I woke up this morning to my wife telling me s...,294,It is soooo awesome. We have been wanting a b...,5|5|5_5|5|5


In [5]:
selfeval = train['selfeval']
train["empathy"] = selfeval.str.extract('\d\|\d\|\d_(\d)\|\d\|\d')
print(train.shape)
train = train.dropna()
print(train.shape)

(79159, 8)
(78304, 8)


In [18]:
conversations = train["conv_id"].value_counts()
print(conversations)
lsconv = pd.DataFrame()
for ids in conversations.index:
    convo = train.loc[train["conv_id"] == ids]
    convo = convo.drop(columns = ["utterance_idx", "selfeval", "conv_id"])
    if len(convo["speaker_idx"].unique()) !=2:
            continue
    s1id = convo["speaker_idx"].unique()[0]
    s2id = convo["speaker_idx"].unique()[1]
    s1 = convo.loc[convo["speaker_idx"]==s1id]
    s2 = convo.loc[convo["speaker_idx"]==s2id]
    s1words = s1["utterance"].values.tolist()
    s2words = s2["utterance"].values.tolist()
    context = convo["context"].str.extract('(\w+)').values[0].item()
    escore = int(convo["empathy"].values[0])
    lsconv = lsconv.append({"context":context, "s1": s1words, "s2": s2words, "e": escore}, ignore_index = True)

hit:2120_conv:4240      8
hit:10149_conv:20299    8
hit:1800_conv:3600      8
hit:6742_conv:13485     8
hit:7143_conv:14287     8
                       ..
hit:9417_conv:18835     1
hit:1412_conv:2825      1
hit:9511_conv:19023     1
hit:4597_conv:9195      1
hit:805_conv:1610       1
Name: conv_id, Length: 18208, dtype: int64


In [16]:
print(lsconv)

         context    e                                                 s1  \
0        hopeful  5.0  [hi there?, did you see my post above about al...   
1        content  5.0  [I love walking my dogs on these late summer m...   
2            sad  5.0  [Leaving home makes me feel unhappy, I have to...   
3        furious  5.0  [Can't believe my professor marked all this wr...   
4          proud  5.0  [Yesterday I bought my very first videogame wi...   
..           ...  ...                                                ...   
195      jealous  4.0  [My friend got a huge scholarship when he bare...   
196      furious  5.0  [I am the only one who cleans in my apartment,...   
197        angry  5.0  [I applied for a job recently. It was a federa...   
198    disgusted  5.0  [My food order today did not turn out as well ...   
199  sentimental  5.0  [I kept all of my kids first day pics every ye...   

                                                    s2  
0    [Hi friend_comma_ how are

My goal is to output a list of lists of the prompt and all that the speaker said, the listener responses, and the first empathy value

First need to take each conversation and seperate it, then need to seperate by speaker_id, and add each response to a new list